In [1]:
import importlib
%load_ext autoreload

import sys
import os
sys.path.append(os.path.abspath("../.."))

import mypackage
import mypackage.elastic
import mypackage.sentence
import mypackage.clustering

from sentence_transformers import SentenceTransformer

2025-05-10 20:58:36.488741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746899916.504142   22917 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746899916.508801   22917 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#doc = mypackage.elastic.Document.from_json("../cached_docs/pubmed_0001.json", text_path="article")
doc = mypackage.elastic.ElasticDocument(
    mypackage.elastic.Session("pubmed-index", cache_dir="../../usercode/cache", use="cache"),
    2581,
    text_path="article"
)

In [5]:
sentences = mypackage.sentence.doc_to_sentences(doc, model)
pair = mypackage.sentence.SimilarityPair.from_sentences(sentences[0], sentences[1])
merged = mypackage.sentence.iterative_merge(sentences, threshold=0.6, round_limit=None, pooling_method="average")
print(merged[0])

["a review of the studies published in the last three decades between 1980 and 2013 reporting on prevalence of childhood overweight and obesity ( age 1 - 18 yr ) in india was conducted using a systematic approach ."]


In [ ]:
%autoreload 2

print(len(merged))

labels, clusters = mypackage.clustering.chain_clustering(merged, n_components=25)
print(len(clusters))

In [ ]:
mypackage.clustering.visualize_clustering(merged, labels)

In [ ]:
from rich.panel import Panel
from rich.console import Console

console = Console()

for cluster in clusters.items():
    text = "\n\n".join([chain.text for chain in cluster[1]])
    console.print(Panel(text, title = f"Cluster {cluster[0]:02}",border_style="cyan bold"))

In [ ]:
print(clusters[1].kth_most_similar_chain().text)

In [ ]:
mask = mypackage.clustering.cluster_mask(clusters)

In [ ]:
print(mask)

In [ ]:
from matplotlib import pyplot as plt

plt.step(list(range(len(mask))), mask)
plt.show()

We need to evaluate the clustering

In [ ]:
import mypackage.clustering.metrics
importlib.reload(mypackage.clustering.metrics)
mypackage.clustering.metrics.chain_clustering_silhouette_score(merged, labels)